In [24]:
# !pip install -q plotly

In [25]:
# from google.colab import drive
# import sys


# drive.mount('/content/drive')
# sys.path.append("/content/drive/MyDrive/second_year_semester_B/Cognitive_of_Animal/collective-motion")


In [26]:
from tools import *

In [27]:
import timeit
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

import pandas as pd
import numpy as np



In [28]:
from typing import Tuple, Callable, Union, List, Optional, Any

Opt = Optional
NumType = Union[int, float]
TensorType = torch.Tensor
BoxDimType = Tuple[NumType, NumType]
UpdateRatioType = Tuple[float,...]
IndScalarType = Union[int, TensorType]
UpdateFuncType = Callable[[TensorType,...], TensorType] 
IndexsType = Union[TensorType, List[IndScalarType], Tuple[IndScalarType, ...], range]
OptimazerType = Union[optim.Optimizer, optim.Adam, optim.SGD, optim.RMSprop]
LossType = Union[nn.Module, nn.CrossEntropyLoss, nn.MSELoss, nn.BCELoss]

<div dir="rtl" lang="he" xml:lang="he">

# מודלים טובים ומודלים טובים יותר


כאשר אנחנו חושבים על מודלים בקוגניציה, אנחנו חושבים על מודלים שמסבירים תופעה במונחים של קוגניציה. 
לדוגמה מודל 
_vicsek_
מסביר את בעלי החיים נעים בלהקה, ע"י מערכת קשרים בן הצורך לשמור על מרחק מהשני, הצורך להתאים את המהירות למהירות של השכנים וכו'.
הבעיה במודלים כאלה היא שהם כמעט לא ניתנים לבדיקה, הרעש גדול מידי. 
אם ננסה לבדוק את ה
_loss_ 
של פונק

תנועה קולקטיבית היא אחד התופעות המענייונות ביותר,
קבוצה של בעלי חיים נעה במרחב בצורה כאילו מאורגנת ללא שוב מנהיג נראה לעין.
אלו מודלים יכולים להסביר את התנועה הזאת?
והאם נמצא דרך להשוות בן המודלים האלו?

<div dir="rtl" lang="he" xml:lang="he">

## הדאטה איתו נעבוד
נשתמש בדאטה של תנועת זגי זברה.
זה יאפשר לנו לבדוק את התנועה במישור, ולא ב
3D
, במרחב
שכמו שזה קורה במציאות. 

In [29]:
# data is a tensor of shape (t,m,2)
# where t is the number of time steps and m is the number of objects.
# The last dim is (x,y) cordination in the plane.
data = torch.load("dataset/60/1.pt")
# using the cordination we can add for every point the speed, acceleration, and angle.
# so now data is a tensor of shape (t,m,5) where for every point we have (x,y, speed, acceleration,angle)
data = add_parameters(data)
# now we can plot the data, we will plot the first 40 steps.
fig = plot_timeline_with_direction(data, "zebra fish 1", steps=range(40))
fig.show()

<div dir="rtl" lang="he" xml:lang="he">

יש סדר מסויים בתנועה, אבל היא מורעשת מאוד, 
ננסה לראות מה מודל תנועה היה נותר למצב כזה. 

<div dir="rtl" lang="he" xml:lang="he">

## מודל פשוט - vicsek
המודל הבסיסי ביותר, הוא המודל של
vicsek
כל בעל חיים מתואר ע"י מיקומו,
מסומן ב
$\vec{r}_i \in R^2$
ווקטור הכיוון שלו שמסומן ב
$\vec{v}_i \in R^1$
כאשר כלל העידכון הוא הליכה בכיון הממוצע.
ע"י חישוב וקטור הכיון של השכנים, מתואר ע"י המשוואה
$$
\\ \vec{S}_i = \sum_{j\in \text{Neg}(i)}{\vec{v}_j}
\\ \vec{v}_i = \frac{\vec{S}_i}{|\vec{S}_i|}*v_0
$$
כאשר
$v_0$
הוא גודל קבוע המייצג את מהירות בעל החיים.




$$
\\ \vec{S} = \sum_{j\in \text{Neg}(i)}{\vec{v}_j}
\\ \vec{v}_i = \frac{\vec{S}}{|\vec{S}|}*v_0
$$

In [30]:
def vicsek_update(
    data: TensorType,
    radius: float = 500,
    update_ratio: UpdateRatioType = (0.7, 0.7, 0.7),
) -> TensorType:
    """update the data according to vicsek model

    Args:
        data (TensorType): tensor (N,5) of x,y,z,theta,phi
        radius (float, optional): radius which one neighbors.Defaults to 500. in pixsels.
        update_ratio (UpdateRatioType, optional): ratio of the update for angle, speed, and. Defaults to (1.0, 1.0, 1.0).

    Returns:
        TensorType: tensor (N,5) new data
    """
    # get the neighbors matrix, is matrix (N,N) where each row is the neighbors of the data point
    # so data[neighbors[i]] is the neighbors of data[i]
    neighbors = get_index_neighbors(data[:, :2], radius)
    update_ratio_angle, update_ratio_speed, update_ratio_acceleratoin = update_ratio
    # get the avrage angle of the neighbors of each data point
    avrage_angle = torch.stack(
        [
            update_ratio_angle * data[neighbors[i], 4].mean(dim=0)
            + (1 - update_ratio_angle) * data[i, 4]
            for i in range(len(data))
        ]
    )
    # get the avrage velocity of the neighbors of each data point
    avrage_speed = torch.stack(
        [
            update_ratio_speed * data[neighbors[i], 2].mean(dim=0)
            + (1 - update_ratio_speed) * data[i, 2]
            for i in range(len(data))
        ]
    )
    avrage_acceleration = torch.stack(
        [
            update_ratio_acceleratoin * data[neighbors[i], 3].mean(dim=0)
            + (1 - update_ratio_acceleratoin) * data[i, 3]
            for i in range(len(data))
        ]
    )

    # update the coordinates
    new_x = avrage_speed * torch.cos(avrage_angle) + data[:, 0]
    new_y = avrage_speed * torch.sin(avrage_angle) + data[:, 1]
    return torch.column_stack(
        [new_x, new_y, avrage_speed, avrage_acceleration, avrage_angle]
    )

In [36]:
ret = create_timeline_series(data[:1], vicsek_update, 100, radius=300)

In [37]:
plot_timeline_with_direction(ret, "test").show()

In [ ]:
p

<div dir="rtl" lang="he" xml:lang="he">

## למידת הפרמטרים של המודל ע"י מחשב

<div dir="rtl" lang="he" xml:lang="he">

## מודלים יותר מסובכים

<div dir="rtl" lang="he" xml:lang="he">

## השוואה בן המודלים ע"י מחשב